In [ ]:
import pandas as pd
import torch
data_train = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/train.csv.gz', compression='gzip',header = None)
data_valid = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/valid.csv.gz', compression='gzip',header = None)
data_test = pd.read_csv('/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/split/time/paper/test.csv.gz', compression='gzip',header = None)
data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/dataset/ogbn_mag/processed/geometric_data_processed.pt")

# Extract edges for "paper" -> "cites" -> "paper"
paper_c_paper = data.edge_index_dict[('paper', 'cites', 'paper')]

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_c_paper[0], nums_train) | torch.isin(paper_c_paper[1], nums_train)
mask_valid = torch.isin(paper_c_paper[0], nums_valid) | torch.isin(paper_c_paper[1], nums_valid)
mask_test = torch.isin(paper_c_paper[0], nums_test) | torch.isin(paper_c_paper[1], nums_test)

paper_c_paper_train = paper_c_paper.clone()
paper_c_paper_valid = paper_c_paper.clone()
paper_c_paper_test = paper_c_paper.clone()

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_c_paper_train = paper_c_paper_train[:, mask_train_done]
paper_c_paper_valid = paper_c_paper_valid[:, mask_valid_done]
paper_c_paper_test = paper_c_paper_test[:, mask_test]

len(paper_c_paper_train[1]) + len(paper_c_paper_valid[1]) + len(paper_c_paper_test[1]), paper_c_paper.shape[1]


/tmp/ipykernel_37979/280286001.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data, _ = torch.load(r"/mnt/c/Users/Bruger/Desktop/Bachelor/GraphML_Bachelorprojekt/datase

(5416271, 5416271)

In [23]:
author_w_paper = data.edge_index_dict[('author', 'writes', 'paper')]

author_w_paper_train = author_w_paper.clone()
author_w_paper_valid = author_w_paper.clone()
author_w_paper_test = author_w_paper.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(author_w_paper[1], nums_train)
mask_valid = torch.isin(author_w_paper[1], nums_valid)
mask_test = torch.isin(author_w_paper[1], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
author_w_paper_train = author_w_paper_train[:, mask_train_done]
author_w_paper_valid = author_w_paper_valid[:, mask_valid_done]
author_w_paper_test = author_w_paper_test[:, mask_test]

len(author_w_paper_train[1]) + len(author_w_paper_valid[1]) + len(author_w_paper_test[1]), author_w_paper.shape[1]


(7145660, 7145660)

In [25]:
paper_t_field = data.edge_index_dict[('paper', 'has_topic', 'field_of_study')]

paper_t_field_train = paper_t_field.clone()
paper_t_field_valid = paper_t_field.clone()
paper_t_field_test = paper_t_field.clone()

# Unique paper IDs to keep (Ensure it's a PyTorch tensor)
nums_valid = torch.tensor(data_valid[0])
nums_test = torch.tensor(data_test[0])
nums_train = torch.tensor(data_train[0])

mask_train = torch.isin(paper_t_field[0], nums_train)
mask_valid = torch.isin(paper_t_field[0], nums_valid)
mask_test = torch.isin(paper_t_field[0], nums_test)

# Combine the conditions into a single mask that selects only the train edges
mask_train_done = mask_train & ~mask_valid & ~mask_test
mask_valid_done = mask_valid & ~mask_test

# Apply the combined mask to paper_c_paper_train
paper_t_field_train = paper_t_field_train[:, mask_train_done]
paper_t_field_valid = paper_t_field_valid[:, mask_valid_done]
paper_t_field_test = paper_t_field_test[:, mask_test]

len(paper_t_field_train[1]) + len(paper_t_field_valid[1]) + len(paper_t_field_test[1]), paper_t_field.shape[1]


(7505078, 7505078)

In [24]:
data

Data(
  num_nodes_dict={
    author=1134649,
    field_of_study=59965,
    institution=8740,
    paper=736389,
  },
  edge_index_dict={
    (author, affiliated_with, institution)=[2, 1043998],
    (author, writes, paper)=[2, 7145660],
    (paper, cites, paper)=[2, 5416271],
    (paper, has_topic, field_of_study)=[2, 7505078],
  },
  x_dict={ paper=[736389, 128] },
  node_year={ paper=[736389, 1] },
  edge_reltype={
    (author, affiliated_with, institution)=[1043998, 1],
    (author, writes, paper)=[7145660, 1],
    (paper, cites, paper)=[5416271, 1],
    (paper, has_topic, field_of_study)=[7505078, 1],
  },
  y_dict={ paper=[736389, 1] }
)